# Plot utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML
from scipy.interpolate import interp1d

def draw_bin(x):
    xmin = x - .5
    xmax = x + .5
    plt.plot([xmin, xmax], [0, 0], c='k', lw=3)
    plt.plot([xmin, xmin], [0, 1], c='k', lw=3)
    plt.plot([xmax, xmax], [0, 1], c='k', lw=3)

def draw_worker(x, y):
    plt.plot([x, x], [y, 2], c='b', lw=5)
    plt.scatter(x, y + .02, marker='^', c='b', s=200, zorder=2)

box_width = .5
box_height = .25
def draw_box(x, y):
    box = Rectangle([x - box_width / 2, y], box_width, box_height, color='sandybrown', ec='k')
    plt.gca().add_patch(box)

def draw_scene(worker, box):
    draw_bin(0)
    draw_bin(2)
    draw_worker(*worker)
    draw_box(*box)

def subsample(workers, boxes, n=100):
    dists = np.linalg.norm(workers[1:] - workers[:-1], axis=1)
    dists = np.insert(dists, 0, 0)
    knots = np.cumsum(dists) / sum(dists)
    workers_fun = interp1d(knots, workers, axis=0)
    boxes_fun = interp1d(knots, boxes, axis=0)
    t = np.arange(n) / (n - 1)
    return workers_fun(t), boxes_fun(t)

def animate(workers, boxes, n=100):
    assert len(workers) == len(boxes)
    workers, boxes = subsample(workers, boxes)
    fig, ax = plt.subplots()
    def draw_frame(i):
        plt.clf()
        draw_scene(workers[i], boxes[i])
    ani = FuncAnimation(fig, draw_frame, frames=len(workers), interval=1e2)
    display(HTML(ani.to_jshtml()))
    plt.close()

In [ ]:
workers = np.array([[1, 1.5], [0, 1.5], [0, .25], [0,  1.5], [2,  1.5], [2, .25], [2, 1.5], [1, 1.5]])
boxes   = np.array([[0,   0], [0,   0], [0,   0], [0, 1.25], [2, 1.25], [2,   0], [2,   0], [2,   0]])
animate(workers, boxes)

# Skills

- **move**: point to point motion on top of the bins
- **pick_i**: picks object in ith bin
- **place_i**: places object in ith bin

In [ ]:
def add_boundary_states(v):
    return [v.add_variable(2) for _ in range(4)]

def move(gcs, label, grasp):
    assert grasp in [0, 1]
    v = gcs.add_vertex('move_' + label)
    w_init, b_init, w_term, b_term = add_boundary_states(v)
    v.add_cost(cp.norm2(w_term - w_init))
    for b in [b_init, b_term]:
        v.add_constraint(b >= [-.5 + box_width / 2, 0])
        v.add_constraint(b <= [2.5 - box_width / 2, 2 - box_height])
    for w in [w_init, w_term]:
        v.add_constraint(w >= [-.5, 1 + box_height * grasp])
        v.add_constraint(w <= [2.5, 2])
    v.add_constraint(b_term - b_init == (w_term - w_init) * grasp)
    return v

def pick(gcs, i):
    v = gcs.add_vertex(f'pick_{i}')
    w_init, b_init, w_term, b_term = add_boundary_states(v)
    w_pick = v.add_variable(2)
    v.add_cost(cp.norm2(w_pick - w_init))
    v.add_cost(cp.norm2(w_term - w_pick))
    v.add_constraint(b_init >= [-.5 + box_width / 2 + 2 * i, 0])
    v.add_constraint(b_init <= [.5 - box_width / 2 + 2 * i, 0])
    v.add_constraint(b_term >= [-.5 + box_width / 2 + 2 * i, 1])
    v.add_constraint(b_term <= [.5 - box_width / 2 + 2 * i, 1])
    v.add_constraint(w_init >= [-.5 + 2 * i, 0])
    v.add_constraint(w_init <= [.5 + 2 * i, 1])
    v.add_constraint(w_term >= [-.5 + 2 * i, 1 + box_height])
    v.add_constraint(w_term <= [.5 + 2 * i, 1 + box_height])
    v.add_constraint(w_pick[0] >= b_init[0] - box_width / 2)
    v.add_constraint(w_pick[0] <= b_init[0] + box_width / 2)
    v.add_constraint(w_pick[1] == box_height)
    v.add_constraint(w_term - b_term == w_pick - b_init)
    return v

def place(gcs, i):
    v = gcs.add_vertex(f'place_{i}')
    w_init, b_init, w_term, b_term = add_boundary_states(v)
    w_place = v.add_variable(2)
    v.add_cost(cp.norm2(w_place - w_init))
    v.add_cost(cp.norm2(w_term - w_place))
    v.add_constraint(b_init >= [-.5 + box_width / 2 + 2 * i, 1])
    v.add_constraint(b_init <= [.5 - box_width / 2 + 2 * i, 1])
    v.add_constraint(b_term >= [-.5 + box_width / 2 + 2 * i, 0])
    v.add_constraint(b_term <= [.5 - box_width / 2 + 2 * i, 0])
    v.add_constraint(w_init >= [-.5 + 2 * i, 1 + box_height])
    v.add_constraint(w_init <= [.5 + 2 * i, 1 + box_height])
    v.add_constraint(w_term >= [-.5 + 2 * i, 0])
    v.add_constraint(w_term <= [.5 + 2 * i, 1])
    v.add_constraint(w_place[0] >= b_term[0] - box_width / 2)
    v.add_constraint(w_place[0] <= b_term[0] + box_width / 2)
    v.add_constraint(w_place[1] == box_height)
    v.add_constraint(w_place - b_term == w_init - b_init)
    return v

def add_edge(gcs, v, w):
    e = gcs.add_edge(v, w)
    e.add_constraint(v.variables[2] == w.variables[0])
    e.add_constraint(v.variables[3] == w.variables[1])
    return e

# GCS

In [ ]:
initial_state = np.array([[2, 1.5], [0, 0]])
final_state_min = np.array([[0, 1.5], [1.5 + box_width / 2, 0]])
final_state_max = np.array([[0, 1.5], [2.5 - box_width / 2, 0]])

In [ ]:
import cvxpy as cp
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

s = gcs.add_vertex('s')
w_init, b_init, w_term, b_term = add_boundary_states(s)
s.add_constraint(w_init == initial_state[0])
s.add_constraint(b_init == initial_state[1])
s.add_constraint(w_term == w_init)
s.add_constraint(b_term == b_init)

t = gcs.add_vertex('t')
w_init, b_init, w_term, b_term = add_boundary_states(t)
t.add_constraint(w_term == w_init)
t.add_constraint(b_term == b_init)
t.add_constraint(w_term >= final_state_min[0])
t.add_constraint(b_term >= final_state_min[1])
t.add_constraint(w_term <= final_state_max[0])
t.add_constraint(b_term <= final_state_max[1])

move0 = move(gcs, '0', grasp=0)
pick_left = pick(gcs, i=0)
pick_right = pick(gcs, i=1)

add_edge(gcs, s, move0)
add_edge(gcs, s, pick_left)
add_edge(gcs, s, pick_right)
add_edge(gcs, move0, pick_left)
add_edge(gcs, move0, pick_right)

move1 = move(gcs, '1', grasp=1)
add_edge(gcs, pick_left, move1)
add_edge(gcs, pick_right, move1)

place_left = place(gcs, i=0)
place_right = place(gcs, i=1)

add_edge(gcs, move1, place_left)
add_edge(gcs, move1, place_right)

move2 = move(gcs, '2', grasp=0)
add_edge(gcs, place_left, move2)
add_edge(gcs, place_right, move2)

add_edge(gcs, move2, t)
add_edge(gcs, place_left, t)
add_edge(gcs, place_right, t)

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_shortest_path(s, t)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
path = [s]
workers = []
boxes = []
while path[-1] != t:
    for e in gcs.outgoing_edges(path[-1]):
        ye = e.y.value
        if ye is not None and ye > .5:
            v = e.head
            path.append(v)
            workers.append(v.variables[0].value)
            boxes.append(v.variables[1].value)
            if v.name[:4] == 'pick':
                workers.append(v.variables[-1].value)
                boxes.append(v.variables[1].value)
            if v.name[:5] == 'place':
                workers.append(v.variables[-1].value)
                boxes.append(v.variables[3].value)
            workers.append(v.variables[2].value)
            boxes.append(v.variables[3].value)
            break
workers = np.array(workers)
boxes = np.array(boxes)

In [ ]:
animate(workers, boxes)